In [1]:
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Embedding, TimeDistributed
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Attention, Concatenate
from tensorflow import keras
from tensorflow.keras import layers
from keras_contrib.layers import CRF

2022-12-13 15:40:36.922212: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow
from numpy.random import seed
tensorflow.random.set_seed(2022)
seed(2022)

In [3]:
import pandas as pd
import numpy as np
import string
import spacy
from keras.preprocessing.text import Tokenizer 
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS
from heapq import nlargest

This is a word contraction dictionary I got from: https://www.kaggle.com/code/singhabhiiitkgp/text-summarization-using-lstm, I use this to help me remove contracted words and replace them with common words.

In [4]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}


In [5]:
def get_data(path):
    return pd.read_csv(path)

data = get_data("/Users/williamlee/Desktop/CS505/Final Project/Reviews.csv")

In [6]:
X_train = data["Text"]
y_train = data["Summary"]

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/williamlee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# #determine outliers by length
# #I am using mean and standard deviatoion to determine which sentences are outliers

# length_text = []
# length_summary = []
# for x in range(len(X_train)):
#     length_text.append(len(X_train[x].split()))
#     length_summary.append(len(y_train[x].split()))
    
# def cal_mean_and_std(test_list):
#     mean = sum(test_list) / len(test_list)
#     variance = sum([((x - mean) ** 2) for x in test_list]) / len(test_list)
#     res = variance ** 0.5
#     return mean, res

In [9]:
# X_mean, X_std = cal_mean_and_std(length_text)

In [10]:
# y_mean, y_std = cal_mean_and_std(length_summary)

In [11]:
# new_texts = []
# new_summaries = []
# for i in range(len(X_train)):
#     if len(X_train[i].split())<=X_mean+3*X_std and len(y_train[i].split())<=y_mean+3*y_std:
#         new_texts.append(X_train[i])
#         new_summaries.append(y_train[i])

In [12]:
# #We have to add some words to indicate the start and end of the summaries
# #I am not sure why people use sostok and eostok, but I notice that almost everyone use these, so 
# #I am going to use the same

# new_df = pd.DataFrame({'text': new_texts,'summary': new_summaries})
# new_df['summary'] = new_df['summary'].apply(lambda x: 'sostok ' + x + ' eostok')

In [13]:
# #split the data into training data and testing data
# from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test=train_test_split(new_df['text'],new_df['summary'],test_size=0.1,random_state=2022,shuffle=True) 

In [14]:
# #here, we are counting the number of words that appears less than 5 times
# #Then, we are going to use that count to initialize a new tokenizer, which we neglect words that appears less than 5 times

# def tokenize(X_train, X_test):
#     x_tokenizer = Tokenizer() 
#     x_tokenizer.fit_on_texts(list(X_train))
    
#     val = 5
#     count = 0
#     total_count = 0

#     for key, value in x_tokenizer.word_counts.items():
#         total_count += 1
#         if value < val:
#             count += 1

#     count = total_count - total_count
#     tokenizer = Tokenizer(num_words = count)
#     inp = list(X_train)
#     tokenizer.fit_on_texts(inp)
    
#     max_len = int(X_mean+3*X_std)
#     train_seq = tokenizer.texts_to_sequences(X_train) 
#     train = pad_sequences(train_seq,  maxlen=max_len, padding='post')
    
#     test_seq = tokenizer.texts_to_sequences(X_test)
#     test = pad_sequences(test_seq, maxlen=max_len, padding='post')
    
#     voc =  tokenizer.num_words + 1
    
#     return train, test, voc, tokenizer


In [15]:
# X_train, X_test, X_voc, X_tokenizer = tokenize(X_train, X_test)

In [16]:
# y_train, y_test, y_voc, y_tokenizer = tokenize(y_train, y_test)

In [17]:
# #we are going to delete the samples with empty summaries

# def delete_empty(x, y):
#     ind=[]
#     for i in range(len(y)):
#         count=0
#         for j in y[i]:
#             if j!=0:
#                 count+=1
#         if count==2:
#             ind.append(i)
    
# #     y_set = []
# #     x_set = []
# #     for i in range(len(x)):
# #         if i not in ind:
# #             x_set.append(x[i])
# #             y_set.append(y[i])
            
# #     x_set = np.array(x_set)
# #     y_set = np.array(y_set)
    
#     x_set=np.delete(x,ind, axis=0)
#     y_set=np.delete(y,ind, axis=0)
    
    
#     return x_set, y_set

In [18]:
# X_train, y_train = delete_empty(X_train, y_train)
# X_test, y_test = delete_empty(X_test, y_test)

In [19]:
# X_train.shape, y_train.shape

In [20]:
# X_test.shape, y_test.shape

In [21]:
#Define our LSTM model

# n_tags = 5022

# input = Input(shape=(int(X_mean+3*X_std),))
# model = Embedding(input_dim=17512, output_dim=5000, trainable=True)(input)  
# model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.2))(model)
# model = TimeDistributed(Dense(5022, activation="softmax"))(model)
# crf = CRF(n_tags+1)
# out = crf(model)

# model = Model(input, out)
# model.summary()

I got this inspiration from the following websites and articles:<br>
https://www.researchgate.net/publication/348531563_Text_Summarization_Using_Text_Frequency_Ranking_Sentence_Prediction<br>
https://aip.scitation.org/doi/pdf/10.1063/5.0037283<br>
https://medium.com/@ashins1997/text-summarization-e702168569f6<br>
https://ijisrt.com/wp-content/uploads/2019/07/IJISRT19JU589.pdf


In [22]:
def summarizarion_freq(text, percentage):
    #perform some tokenization, from assginments
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
#     print()
#     print(tokens)
#     print()
    for index in range(len(tokens)):
        if tokens[index] in contraction_mapping:
            tokens[index]=contraction_mapping[tokens[index]]
    text = ' '.join(tokens)
#     print(text)
    doc= nlp(text)
            
    
    #Next we are going to find the frequencies of tokens
    freq={}
    for word in doc:
        if word.text not in list(STOP_WORDS) and word.text not in punctuation:
            if word.text not in freq.keys():
                freq[word.text]=1
            else:
                freq[word.text]+=1
                
    #get the maximum frequency
    max_freq = max(list(freq.values()))
    
    for key in freq.keys():
        freq[key] = freq[key]/max_freq
    
    #we are going to compute the score, frequency sum of each sentence
    sent_tokens = [sentence for sentence in doc.sents]
    scores = {}
    for sent in sent_tokens:
        for word in sent:
            if word.text.lower() in freq:
                if sent not in scores:
                    scores[sent] = freq[word.text.lower()]
                else:
                    scores[sent] += freq[word.text.lower()]
    
    # We are going to pick the sentences with the top priorities with given percentage
    select_len = int(len(sent_tokens)*percentage)
    summary = nlargest(select_len, scores, key=scores.get)
#     print()
#     print(scores)
#     print()
#     print(summary)
#     print()
    summary = [w.text for w in summary]
#     print()
#     print(summary)
#     print()
    summary = ''.join(summary)
    summary = summary.replace("\n", "")
    summary = summary.replace(" '", "'")
    summary = summary.replace(" ,", ",")
    summary = summary.replace(" .", ".")
    
    return summary

Let's first test the summarizer on a long text.

In [23]:
my_file = open("/Users/williamlee/Desktop/CS505/Final Project/dataset/DUC2004/DUC2004_Summarization_Documents/duc2004_testdata/tasks1and2/duc2004_tasks1and2_docs/docs/1/D1.txt")
text = my_file.read()

In [24]:
text

"\nCambodian leader Hun Sen on Friday rejected opposition parties' demands \nfor talks outside the country, accusing them of trying to ``internationalize'' \nthe political crisis. Government and opposition parties have asked \nKing Norodom Sihanouk to host a summit meeting after a series of post-election \nnegotiations between the two opposition groups and Hun Sen's party \nto form a new government failed. Opposition leaders Prince Norodom \nRanariddh and Sam Rainsy, citing Hun Sen's threats to arrest opposition \nfigures after two alleged attempts on his life, said they could not \nnegotiate freely in Cambodia and called for talks at Sihanouk's residence \nin Beijing. Hun Sen, however, rejected that. ``I would like to make \nit clear that all meetings related to Cambodian affairs must be conducted \nin the Kingdom of Cambodia,'' Hun Sen told reporters after a Cabinet \nmeeting on Friday. ``No-one should internationalize Cambodian affairs. \nIt is detrimental to the sovereignty of Camb

In [25]:
summary = summarizarion_freq(text, 0.2)
print(summary)

Opposition leaders Prince Norodom  Ranariddh and Sam Rainsy, citing Hun Sen's threats to arrest opposition  figures after two alleged attempts on his life, said they could not  negotiate freely in Cambodia and called for talks at Sihanouk's residence  in Beijing.Government and opposition parties have asked  King Norodom Sihanouk to host a summit meeting after a series of post - election  negotiations between the two opposition groups and Hun Sen's party  to form a new government failed. Cambodian leader Hun Sen on Friday rejected opposition parties' demands  for talks outside the country, accusing them of trying to ` ` internationalize''  the political crisis.


In [26]:
sample_summary = open("/Users/williamlee/Desktop/CS505/Final Project/dataset/DUC2004/reference/Task1_reference1.txt").read()
print(sample_summary)

Prospects were dim for resolution of the political crisis in Cambodia in October 1998.
Prime Minister Hun Sen insisted that talks take place in Cambodia while opposition leaders Ranariddh and Sam Rainsy, fearing arrest at home, wanted them abroad.
King Sihanouk declined to chair talks in either place.
A U.S. House resolution criticized Hun Sen's regime while the opposition tried to cut off his access to loans.
But in November the King announced a coalition government with Hun Sen heading the executive and Ranariddh leading the parliament.
Left out, Sam Rainsy sought the King's assurance of Hun Sen's promise of safety and freedom for all politicians.



Let's see how this frequency summarizer handles short texts.

In [27]:
X_train[0]

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [28]:
print(summarizarion_freq(X_train[0], 0.5))

The product looks more like a stew than a processed meat and it smells better.


In [29]:
print(y_train[0])

Good Quality Dog Food


Let's see how well T5 Transformer summarizes long and short texts.<br>
Reference: https://huggingface.co/transformers/v3.0.2/model_doc/t5.html<br>
Result: T5 does a bad work for both long and shor texts, they miss the important sentences. In the long text, there is no mention of who and where. For the short text, the thesis is that the dog food has good qulity, but there is no mention of that.

In [30]:
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration

my_model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
concat_text = "summarize:" + text
input=tokenizer.encode(concat_text, return_tensors='pt', max_length=80)
summary = my_model.generate(input)
summary_T5 = tokenizer.decode(summary[0])
print(summary_T5)

/Users/williamlee/opt/anaconda3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode

<pad> government and opposition parties have asked the opposition to host a summit. the two opposition groups


In [31]:
my_model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
concat_text = "summarize:" + X_train[0]
input=tokenizer.encode(concat_text, return_tensors='pt', max_length=20)
summary = my_model.generate(input)
summary_T5 = tokenizer.decode(summary[0])
print(summary_T5)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<pad> canned dog food products have been found all over the world. i have bought several of


Let's see how well BART Transformer summarizes long and short texts. <br>
Reference: https://huggingface.co/transformers/v2.11.0/model_doc/bart.html <br>
Result: BART does a good work for the long text, but it wasn't able to capture important information such as locations.
    Fro the short text, BART does a really bad work, as it just returns the original text.

In [32]:
from transformers import BartForConditionalGeneration, BartTokenizer

tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
inputs = tokenizer.batch_encode_plus([text],return_tensors='pt')
summary = model.generate(inputs['input_ids'], early_stopping=True)
summary_BART = tokenizer.decode(summary[0], skip_special_tokens=True)
print(summary_BART)

/Users/williamlee/opt/anaconda3/lib/python3.8/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Cambodian leader Hun Sen rejects opposition parties' demands for talks outside the country. Government and opposition parties have asked King Norodom Sihanouk to host a summit meeting. Opposition leaders cite Hun Sen's threats to arrest opposition figures after two alleged attempts on his life. Ranariddh and Sam Rainsy have charged that Hun Sen's victory was achieved through widespread fraud.


In [33]:
tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
inputs = tokenizer.batch_encode_plus([X_train[0]],return_tensors='pt')
summary = model.generate(inputs['input_ids'], early_stopping=True)
summary_BART = tokenizer.decode(summary[0], skip_special_tokens=True)
print(summary_BART)

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


Let's see how well GPT2 Transformer summarizes long and short texts. <br>
Reference: https://huggingface.co/transformers/v3.0.2/model_doc/gpt2.html <br>
Result: GPT2 does a terrible works for long texts, GPT2 doens't really capture all the important things form the original text. For the short text, it does a pretty good work at summarization.

In [34]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel

tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
model=GPT2LMHeadModel.from_pretrained('gpt2')
inputs=tokenizer.batch_encode_plus([text],return_tensors='pt',max_length=120)
summary=model.generate(inputs['input_ids'],early_stopping=True)
summary_GPT=tokenizer.decode(summary[0],skip_special_tokens=True)
print(summary_GPT)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 120, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.



Cambodian leader Hun Sen on Friday rejected opposition parties' demands 
for talks outside the country, accusing them of trying to ``internationalize'' 
the political crisis. Government and opposition parties have asked 
King Norodom Sihanouk to host a summit meeting after a series of post-election 
negotiations between the two opposition groups and Hun Sen's party 
to form a new government failed. Opposition leaders Prince Norodom 
Ranariddh and Sam Rainsy, citing Hun Sen's threats to arrest opposition 
figures


In [35]:
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
model=GPT2LMHeadModel.from_pretrained('gpt2')
inputs=tokenizer.batch_encode_plus([X_train[0]],return_tensors='pt',max_length=10)
summary=model.generate(inputs['input_ids'],early_stopping=True)
summary_GPT=tokenizer.decode(summary[0],skip_special_tokens=True)
print(summary_GPT)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I have bought several of the Vitality canned dog food products and have been very pleased with the quality
